<a href="https://colab.research.google.com/github/dineshkumarsarangapani/Machine_learning_IITM/blob/main/TDS/L6_5_Networks_Kumu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
!pip install scikit-network==0.24.0


In [ ]:
# Download the data
!rm *.tsv.gz
# !wget -q https://datasets.imdbws.com/name.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.akas.tsv.gz
# !wget -q https://datasets.imdbws.com/title.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.akas.tsv.gz
# !ls -la

!wget -q https://datasets.imdbws.com/name.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!wget -q https://datasets.imdbws.com/title.basics.tsv.gz
!wget -q https://datasets.imdbws.com/title.crew.tsv.gz
!wget -q https://datasets.imdbws.com/title.episode.tsv.gz
!wget -q https://datasets.imdbws.com/title.principals.tsv.gz
!wget -q https://datasets.imdbws.com/title.ratings.tsv.gz
!ls -la

In [ ]:
# Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear']]
title.head()

In [ ]:
title = title[title['startYear'].str.find('\\N')==-1]

In [ ]:
title['startYear'] = title.startYear.values.astype('int')

In [ ]:
title = title[title['startYear']>=2005]

In [ ]:
# !rm *.tsv.gz
# !wget -q https://datasets.imdbws.com/name.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.principals.tsv.gz  
# !wget -q https://datasets.imdbws.com/title.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.akas.tsv.gz
# !ls -la

In [ ]:
# Download the data
# !rm *.tsv.gz
# !wget -q https://datasets.imdbws.com/name.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.principals.tsv.gz
# !wget -q https://datasets.imdbws.com/title.basics.tsv.gz
# !wget -q https://datasets.imdbws.com/title.principals.akas.gz
# !wget -q https://datasets.imdbws.com/title.akas.tsv.gz
!ls -la
#Load the titles
title = pd.read_csv('title.basics.tsv.gz', sep='\t', low_memory=False).set_index('tconst')[['titleType', 'primaryTitle', 'startYear','genres']]
title.head()



In [ ]:
genere = title['genres'].unique()
title.shape

In [ ]:
# for g in genere:
  # print(g)

In [ ]:
title = title[title['titleType'] == 'movie']
title = title[title['startYear'].str.find('\\N')==-1]
title['startYear'] = title.startYear.values.astype('int')
title = title[title['startYear']>=2005]
title = title[title['genres'].str.contains('Comedy')]

In [ ]:
len(title)

In [ ]:

title.index

In [ ]:
title.columns

In [ ]:
# from google.colab import files
# title.to_csv('file1.csv')
# files.download('file1.csv') 

In [ ]:
# Load the cast of each film
cast = pd.read_csv('title.principals.tsv.gz', sep='\t', low_memory=False)[['tconst', 'nconst', 'category']]
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
# movies = title[title['titleType'] == 'movie']
# cast = cast[cast['tconst'].isin(movies.index)]
cast.head()

In [ ]:
# Load the cast of each film
cast = pd.read_csv('title.principals.tsv.gz', sep='\t', usecols=['tconst', 'nconst', 'category'], dtype={'tconst':'str', 'nconst':'str', 'category': 'str'}, na_values='\\N').set_index('tconst')
# Only consider actors, not directors, composers, etc. Shrinks data to about 40%
cast = cast[cast.category.isin({'actor', 'actress'})]
# movies = title[title['titleType'] == 'movie']
# cast = cast[cast['tconst'].isin(movies.index)]
cast.head()

In [ ]:
# Only consider movies, not TV series, etc. Shrinks data to ~5%
movies = title[title['titleType'] == 'movie']
# cast = cast[cast['tconst'].isin(movies.index)]
# # This is what the network looks like
# cast.head()

In [ ]:
# Restrict data to just a single region (e.g. IN, US, etc)
# This loads the region for each title


region = pd.read_csv('title.akas.tsv.gz', sep='\t', low_memory=True).set_index('titleId')['region']
region.head(10)
# new_region = region[region['language'] == 'hi']

In [ ]:
new_region=pd.DataFrame(region)

In [ ]:
new_region.head()

In [ ]:
new_region['region'].unique()

In [ ]:

new_region2 = new_region[new_region['region'] == 'US']

In [ ]:
new_region2

In [ ]:
# from google.colab import files
# new_region2.to_csv('file2.csv')
# files.download('file2.csv') 

In [ ]:
# Load the name data along with birth year
name = pd.read_csv('name.basics.tsv.gz', sep='\t', na_values='\\N', dtype={'birthYear': float}).set_index('nconst')[['primaryName', 'birthYear']]

In [ ]:
name.head()

In [ ]:
# from google.colab import files
# name.to_csv('file3.csv')
# files.download('file3.csv') 

In [ ]:
len(name)

In [ ]:
from scipy.sparse import csr_matrix

In [ ]:
def get_pairs(lang='IN', min_acted=1, min_pairings=1):
     '''
#     Returns an adjacency matrix and actor mapping of actor pairs where:
#     - Each actor has acted in at least min_acted films
#     - The two actors have acted together in at least min_pairings films
#     - And (optionally), belong to a region `lang` (IN, UN, etc)
#     '''
     graph = cast
     if lang is 'IN':
        graph = graph[graph['tconst'].isin(region[region == lang].index)]
        pairs = pairs.drop(columns=['row', 'col'])
        pairs.columns = ['count', 'name1', 'year1', 'name2', 'year2']
     return pairs.sort_values('count', ascending=False)

pairs, cat = get_pairs('IN', 1, 1)


In [ ]:
cat

In [ ]:
pairs

In [ ]:
ForKumu = lookup(pairs, cat)
ForKumu

In [ ]:
ForKumu = ForKumu[['name1', 'name2', 'count']]
ForKumu = ForKumu.rename(columns={'name1':'From',
                                  'name2':'To',
                                  'count':'Strength'})
ForKumu

In [ ]:
ForKumu.to_excel("pairs.xlsx", index = False)